## NLP with DL

### Import libraries

In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# RNN de bir nöronun çıkışı diğer nöronun girişi oluyor.(Sıralı bir düzen var)

In [2]:
df = pd.read_csv("hepsiburada.csv")
df

,Rating,Review
0,1,3 yıldır tık demedi. :)
1,1,3 yıldır kullanıyorum müthiş
2,1,Ürün bugün elime geçti çok fazla inceleme fırs...
3,1,Almaya karar verdim. Hemencecik geldi. Keyifle...
4,1,Günlük kullanımınızı çok çok iyi karsılıyor kı...
...,...,...
243492,1,fiyatına göre güzel
243493,1,Ürün kullanışlı iş görüyor fazlasıyla eşime al...
243494,1,"Hızlı Kargo, güzel ürün"
243495,1,telefon başarılı hızlı bir cihaz sadece beyaz...


### Train Test Split

In [3]:
X = df["Review"].values
y = df["Rating"].values
# Modelimizi DL de çalıştıracağımız için array e dönüştürdük.

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=101)

<IPython.core.display.Javascript object>

In [5]:
X_train[251]

'teşekkür ederim memnunum'

In [6]:
y_train[251]

1

### Tokenization

In [7]:
num_words = 15000
tokenizer = Tokenizer(num_words=num_words) # filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n0123456789', bu noktalama ve sayıları datadan çıkarıyor.
# Embedding de CountVectorizer ve TF-IDF deki Lemmatizer vb işlemleri uygulamıyoruz. Çünkü Embedding kelimelerin anlamları olduğu gibi alınır.
# Burada token leri sayısallaştırıyoruz. 

In [8]:
tokenizer.fit_on_texts(X) 

### Creating word index

In [9]:
tokenizer.word_index
# Her bir token e bir sayısal ifade atanıyor en fazla kullanılandan başlayarak (15 000 e kadar 15 000 üzerini göz ardı ediyor).

{'çok': 1,
 'bir': 2,
 've': 3,
 'ürün': 4,
 'bu': 5,
 'iyi': 6,
 'güzel': 7,
 'için': 8,
 'tavsiye': 9,
 'ederim': 10,
 'daha': 11,
 'ama': 12,
 'da': 13,
 'gayet': 14,
 'hızlı': 15,
 'teşekkürler': 16,
 'aldım': 17,
 'de': 18,
 'ürünü': 19,
 'gibi': 20,
 'yok': 21,
 'uygun': 22,
 'olarak': 23,
 'kaliteli': 24,
 'en': 25,
 '2': 26,
 'kargo': 27,
 'fiyat': 28,
 'elime': 29,
 'kadar': 30,
 'ile': 31,
 'göre': 32,
 'geldi': 33,
 'var': 34,
 'hepsiburada': 35,
 'ben': 36,
 'gerçekten': 37,
 '1': 38,
 'fiyata': 39,
 'gün': 40,
 'sonra': 41,
 'cok': 42,
 'kesinlikle': 43,
 'telefon': 44,
 'biraz': 45,
 'hiç': 46,
 'ulaştı': 47,
 'memnun': 48,
 'hem': 49,
 'değil': 50,
 'kullanışlı': 51,
 '3': 52,
 'mükemmel': 53,
 'oldu': 54,
 'kullanıyorum': 55,
 'önce': 56,
 'sipariş': 57,
 'tek': 58,
 'her': 59,
 'bence': 60,
 'harika': 61,
 'kalitesi': 62,
 'bi': 63,
 'ayrıca': 64,
 '5': 65,
 'teşekkür': 66,
 'fiyatı': 67,
 'olması': 68,
 'ne': 69,
 'herkese': 70,
 'bile': 71,
 'uzun': 72,
 'süper': 73,

In [10]:
len(tokenizer.word_index)

217983

### Converting tokens to numeric

In [11]:
X_train_tokens = tokenizer.texts_to_sequences(X_train)

In [12]:
X_train[205]

'General Mobile Discovery 16 GB telefonumda kullanmak için aldım. Zaten telefonun kapasitesi oldukça iyi bu yüzden hafıza kartı almayı pek düşünmemiştim. Ama Kingston olunca Class10 olunca ve bu fiyata olunca kaçırmak istemedim. Real Racing,Deer Hunter gibi işlemciyi yoracak birçok oyunu hiç takılma olmadan oynuyorum. Zaten hafıza kartında Kingston kendini kanıtlamış 2-3 marka arasındadır. İhtiyacı olan mutlaka alsın.'

In [13]:
print(X_train_tokens[205])
# Kelimeleri yukarıda oluşturulan sayısal ifadeler ile gösteriyor.

[5731, 5675, 9696, 688, 340, 2944, 322, 8, 17, 75, 333, 1994, 87, 6, 5, 369, 740, 1156, 246, 247, 3242, 12, 2192, 649, 12258, 649, 3, 5, 39, 649, 6632, 2723, 20, 620, 1879, 46, 1542, 672, 4231, 75, 740, 2192, 539, 2763, 26, 52, 137, 2463, 113, 293, 1254]


In [14]:
X_test_tokens = tokenizer.texts_to_sequences(X_test)

### Maximum number of tokens for all documents

In [24]:
num_tokens = [len(tokens) for tokens in X_train_tokens + X_test_tokens]
num_tokens = np.array(num_tokens)
num_tokens
# Train ve test verilerini birleştirerek herbir yorumdaki token sayısını kontrol ediyoruz.

array([16, 19,  5, ..., 53,  5, 32])

In [16]:
[1,1]+[2,2] # Note

[1, 1, 2, 2]

In [25]:
num_tokens.mean()
# Token sayılarının ortalamasını alıyoruz.

21.36101060793357

In [26]:
num_tokens.max()
# En fazla token(kelime) içeren cümle.

298

### Fixing token counts of all documents (pad_sequences)

In [31]:
max_tokens = np.array(num_tokens).mean()+2*np.array(num_tokens).std()
max_tokens = int(max_tokens)
max_tokens

61

In [32]:
sum(num_tokens<max_tokens)/len(num_tokens)
# Yaklaşık verinin yüzde 96 sının token sayısı 61 den az. Bunu yapmamızın nedeni matris sayını azaltmak (işlem yükünü azaltmak için)

0.9598968365113328

In [33]:
sum(num_tokens<50)/len(num_tokens)

0.9331408600516639

In [34]:
X_train_pad = pad_sequences(X_train_tokens, maxlen=max_tokens)

In [35]:
X_test_pad = pad_sequences(X_test_tokens, maxlen=max_tokens)

In [38]:
X_train_pad.shape

(194797, 61)

In [39]:
X_test_pad.shape

(48700, 61)

In [40]:
X_train_tokens[800]

[19,
 77,
 29,
 1070,
 5,
 385,
 236,
 4933,
 560,
 80,
 4,
 37,
 449,
 287,
 165,
 240,
 1110,
 239,
 222,
 164,
 2,
 700,
 4650,
 1595,
 3,
 104,
 11852,
 8296,
 46,
 6476,
 5427,
 1636,
 45,
 1371,
 524,
 12,
 2676,
 61,
 908,
 1103,
 5830,
 2299,
 1266,
 4314,
 1272,
 2,
 271,
 2248,
 1363,
 603,
 601,
 779,
 5898,
 4650,
 7937,
 9162,
 2,
 18,
 856,
 12,
 6220,
 4650,
 74,
 532,
 7937,
 14826,
 22,
 1847,
 5491,
 545,
 358,
 16]

In [41]:
np.array(X_train_tokens[800])

array([   19,    77,    29,  1070,     5,   385,   236,  4933,   560,
          80,     4,    37,   449,   287,   165,   240,  1110,   239,
         222,   164,     2,   700,  4650,  1595,     3,   104, 11852,
        8296,    46,  6476,  5427,  1636,    45,  1371,   524,    12,
        2676,    61,   908,  1103,  5830,  2299,  1266,  4314,  1272,
           2,   271,  2248,  1363,   603,   601,   779,  5898,  4650,
        7937,  9162,     2,    18,   856,    12,  6220,  4650,    74,
         532,  7937, 14826,    22,  1847,  5491,   545,   358,    16])

In [42]:
X_train_pad
# Butun cümlerl 61 e tamamladı (ekleme (sıfır ile), kırpma ile)
# Ekleme ve kırpmaları cümlenin başından başlayarak yapıyor. Yani bütün cümleleri sona topluyor. 

array([[   0,    0,    0, ...,  179,   65,  511],
       [   0,    0,    0, ...,    1,   66,   10],
       [   0,    0,    0, ...,   32,   14,    7],
       ...,
       [   0,    0,    0, ..., 2912,   21,   81],
       [   0,    0,    0, ...,   85,  112, 4286],
       [   0,    0,    0, ...,  165,  174,  400]], dtype=int32)

### Converting numeric tokens to string

In [ ]:
index = tokenizer.word_index
inverse = dict(zip(index.values(), index.keys()))
inverse

In [ ]:
def tokens_to_string(tokens):
    words = [inverse[token] for token in tokens if token !=0]
    text = " ".join(words)
    return text

In [ ]:
X_train[800]

In [ ]:
tokens_to_string(X_train_tokens[800])

### Modelling

In [43]:
model = Sequential()

In [44]:
embedding_size = 100
# 100 lük vektöre dönüştürdük. Yani 15 000 word 100 lük vektörlere dönüştürüldü.

In [45]:
model.add(Embedding(input_dim = num_words, output_dim = embedding_size, input_length = max_tokens, name = "embedding_layer"))
# num_words = 15 000, max_tokens = 61

In [ ]:
# !pip install -U numpy==1.19.5

In [48]:
model.add(GRU(units=48, return_sequences=True))
model.add(GRU(units=24, return_sequences=True))
model.add(GRU(units=12))
model.add(Dense(1, activation='sigmoid'))
# Not: LSTM ve GRU da activation olarak "relu" kullanılmıyor, sigmoid ve tanh kullanılıyor.

In [47]:
optimizer = Adam(lr=0.004)

/home/kurubal/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [49]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['Recall'])
# Negative yorumlar bizim için önemli olduğu için metrics Recall kullandık. 

In [51]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 61, 100)           1500000   
_________________________________________________________________
gru (GRU)                    (None, 61, 48)            21600     
_________________________________________________________________
gru_1 (GRU)                  (None, 61, 48)            14112     
_________________________________________________________________
gru_2 (GRU)                  (None, 61, 24)            5328      
_________________________________________________________________
gru_3 (GRU)                  (None, 12)                1368      
_________________________________________________________________
dense (Dense)                (None, 1)                 13        
Total params: 1,542,421
Trainable params: 1,542,421
Non-trainable params: 0
______________________________________________

In [50]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor="val_loss", mode="auto", verbose=1, patience=5, restore_best_weights=True)

In [52]:
pd.Series(y_train).value_counts(normalize=True)

1    0.943834
0    0.056166
dtype: float64

In [53]:
weights = {0:95, 1:5}

In [55]:
model.fit(X_train_pad, y_train, epochs=25, batch_size=256, class_weight=weights,
         validation_data=(X_test_pad, y_test), callbacks=[early_stop])

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/25
761/761 [==============================] - 77s 98ms/step - loss: 3.1771 - recall: 0.8615 - val_loss: 0.2695 - val_recall: 0.8842
Epoch 2/25
761/761 [==============================] - 74s 97ms/step - loss: 1.7538 - recall: 0.9302 - val_loss: 0.2605 - val_recall: 0.8914
Epoch 3/25
761/761 [==============================] - 74s 97ms/step - loss: 1.1393 - recall: 0.9569 - val_loss: 0.1786 - val_recall: 0.9488
Epoch 4/25
761/761 [==============================] - 74s 97ms/step - loss: 0.8768 - recall: 0.9679 - val_loss: 0.1924 - val_recall: 0.9501
Epoch 5/25
761/761 [==============================] - 74s 98ms/step - loss: 0.7040 - recall: 0.9742 - val_loss: 0.2141 - val_recall: 0.9450
Epoch 6/25
761/761 [==============================] - 74s 97ms/step - loss: 0.5877 - recall: 0.9801 - val_loss: 0.1849 - val_recall: 0.9554
Epoch 7/25
761/761 [==

### Model evaluation

In [ ]:
model_loss = pd.DataFrame(model.history.history)
model_loss.head()

In [ ]:
model_loss.plot()

In [ ]:
model.evaluate(x_train_pad, np.array(y_train))

In [ ]:
model.evaluate(x_test_pad, np.array(y_test))

In [ ]:
y_pred_train = model.predict(x=x_train_pad)
y_pred_train

In [ ]:
y_pred_tr = np.array([1.0 if p>0.5 else 0.0 for p in y_pred_train])

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, roc_auc_score

print(confusion_matrix(y_train, y_pred_tr))
print("-------------------------------------------------------")
print(classification_report(y_train, y_pred_tr))

In [ ]:
y_pred = model.predict(x=x_test_pad)
y_pred.T

In [ ]:
y_pr = np.array([1.0 if p>0.5 else 0.0 for p in y_pred])

In [ ]:
print(confusion_matrix(y_test, y_pr))
print("-------------------------------------------------------")
print(classification_report(y_test, y_pr))

### Prediction

In [ ]:
incorrect = np.where(y_test != y_pr)
incorrect 

In [ ]:
len(incorrect)

In [ ]:
idx = incorrect[0][0]
idx

In [ ]:
text = X_test[idx]
text

In [ ]:
y_pred[idx]

In [ ]:
tokens = tokenizer.texts_to_sequences(reviews)

In [ ]:
tokens_pad = pad_sequences(tokens, maxlen=max_tokens)
tokens_pad.shape

In [ ]:
model.predict(tokens_pad)